In [66]:
import ast
import json
import random
import unittest

import networkx as nx
from matplotlib import pyplot as plt
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import GroverOperator, MCMT, ZGate, MCXGate
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_state_city
from qiskit_aer import AerSimulator, Aer

from src.algorithms.QAOA.QAOA import convert_qubo_to_ising, qaoa_optimize, qaoa_no_optimization, sample_results
from src.algorithms.VQE.VQE import vqe_optimization
from src.graph import Graph
from src.algorithms.grover import grover
from src.parser.parser import Parser, CodeVisitor, PROBLEMS
from src.problems.Three_SAT import ThreeSat
from src.problems.clique import Clique
from src.problems.factorization import Factor
from src.problems.max_cut import MaxCut
from src.problems.maximal_independent_set import MIS
from src.problems.tsp import TSP
from src.recommender.recommender_engine import recommender, plot_results
from src.reduction import *
from src.sat_to_qubo import *
from src.circuits_library import *
import csv
import ast

In [48]:
parser = Parser(model_path="../../others/saved_models")

file_path = '../parser/data.csv'
first_column = []

# 读取 CSV 文件第一列
with open(file_path, 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row:
            first_column.append(row[0])

num = 0  # 统计 'Unknown' 类型的个数
mis_code = ""
# 遍历每条记录
for item in first_column:
    try:
        # 尝试将字符串中的 \n 等还原成真实代码
        try:
            clean_code = ast.literal_eval(item)
        except Exception:
            clean_code = item.encode().decode('unicode_escape')

        # 使用 parser 解析 clean_code
        problem_type, data = parser.parse(clean_code)

        # 打印结果
        print("→", problem_type, data)

        # 统计 Unknown 类型
        if problem_type == 'Unknown':
            num += 1
        if problem_type == "MIS":
            mis_code = clean_code

    except Exception as e:
        print("解析失败:", clean_code[:60].replace('\n', ' ') + "...")
        print("错误信息:", e)

# 打印 Unknown 的总数量
print("Number of Unknown problems:", num)

→ MIS Graph with 5 nodes and 5 edges
→ MaxCut <src.graph.Graph object at 0x2a6b83190>
→ MaxCut <src.graph.Graph object at 0x2cad8e7c0>
→ MaxCut <src.graph.Graph object at 0x2caa4fdf0>
→ MaxCut <src.graph.Graph object at 0x2caba0130>
→ MaxCut <src.graph.Graph object at 0x2caa4f8e0>
→ MaxCut <src.graph.Graph object at 0x2caba03d0>
→ MaxCut <src.graph.Graph object at 0x2caa4ff10>
→ MaxCut <src.graph.Graph object at 0x2caba02e0>
→ MaxCut <src.graph.Graph object at 0x2afca2850>
→ MaxCut <src.graph.Graph object at 0x2cad69580>
→ MaxCut <src.graph.Graph object at 0x2afcd55e0>
→ MaxCut <src.graph.Graph object at 0x2cad51e50>
→ MIS <src.graph.Graph object at 0x2cad69070>
→ MIS <src.graph.Graph object at 0x2a68efac0>
→ MIS <src.graph.Graph object at 0x2cad51b80>
→ MIS <src.graph.Graph object at 0x2cad691f0>
→ MIS <src.graph.Graph object at 0x2cad51dc0>
→ MIS <src.graph.Graph object at 0x2cad69f40>
→ MIS <src.graph.Graph object at 0x2cad51250>
→ MIS <src.graph.Graph object at 0x2cad692e0>
→ MIS <

In [67]:
from matplotlib import pyplot as plt
print(mis_code)
tag, data = parser.parse(mis_code)
print(tag, data)
mis = PROBLEMS[tag](data.G)  
mis.report_latex()

def min_vc_solver(n, edge_list):
    vertex_cover = set()
    for u, v in edge_list:
        if u not in vertex_cover and v not in vertex_cover:
            vertex_cover.add(u)
            vertex_cover.add(v)
    return vertex_cover

# Input data
n = 4
edge_list = [(0, 1), (1, 2), (2, 3)]
vc = min_vc_solver(n, edge_list)
print(vc)
MIS <src.graph.Graph object at 0x2cad08460>
Starting Independent Set problem report generation with LaTeX formatting...


/Users/mac/Documents/GitHub/C2Q-Parser/venv/lib/python3.9/site-packages/qiskit/visualization/circuit/matplotlib.py:269: UserWarning: Style JSON file 'mpl.json' not found in any of these locations: /Users/mac/Documents/GitHub/C2Q-Parser/venv/lib/python3.9/site-packages/qiskit/visualization/circuit/styles/mpl.json, ~/mpl.json, ~/.qiskit/mpl.json, mpl.json. Will use default style.
  self._style, def_font_ratio = load_style(self._style)


 - Lowest error: Quantinuum H1 from Azure Quantum with a calculated error of 1.96%, time to execute: 12367.058824 seconds and a price of $36500.0. 
 - Lowest time: IQM Garnet from Amazon Braket with a calculated error of 16.38%, time to execute: 0.122 seconds and a price of $87.5. 
 - Lowest price: Rigetti Ankaa-9Q-3 from Azure Quantum with a calculated error of 20.68%, time to execute: 0.129 seconds and a price of $0.17.


/Users/mac/Documents/GitHub/C2Q-Parser/venv/lib/python3.9/site-packages/qiskit/visualization/circuit/matplotlib.py:269: UserWarning: Style JSON file 'mpl.json' not found in any of these locations: /Users/mac/Documents/GitHub/C2Q-Parser/venv/lib/python3.9/site-packages/qiskit/visualization/circuit/styles/mpl.json, ~/mpl.json, ~/.qiskit/mpl.json, mpl.json. Will use default style.
  self._style, def_font_ratio = load_style(self._style)


{'1001': 18437, '1000': 34056, '0111': 34259, '0101': 18653, '1111': 34368, '1101': 34198, '1010': 18528, '0001': 34337, '0011': 34352, '1011': 33873, '1100': 33982, '0100': 34241, '0110': 34234, '0000': 34257, '0010': 34163, '1110': 34062}
PDF report generated in 13.46 seconds.
